---

03/2019 <p style="text-align: right">Anton Panchenko</p>


![Titanic](https://cdn-static.denofgeek.com/sites/denofgeek/files/styles/main_wide/public/2015/10/raise-main.jpg?itok=QBxamb0z)

In [ ]:
import os
import re
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import metrics

os.listdir("../input")

## Introduction

Load and merge datasets, meet data

In [ ]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
data = train.append(test, sort=False)
data.sample(3)

Look at types and incomplete features

In [ ]:
def count(df:pd.DataFrame):
    stat = pd.DataFrame([df.dtypes, df.count()], index=['dtypes', 'count'])
    return stat.sort_values(by=['count'], axis=1, ascending=False)

count(data)

It seems we have 1309 objects. Incomplete features are:
- Fare
- Embarked
- Age
- Cabin

## Create FamilySize and Alone features

In [ ]:
data['FamilySize'] = data['SibSp'] + data['Parch'] + 1

data['Alone'] = 0
data.loc[data['FamilySize'] == 1, 'Alone'] = 1

def value_counts(feature:pd.Series):
    df = pd.DataFrame([feature.value_counts()], index=[feature.name + ' counts'])
    df['nans'] = feature.isna().sum()
    return df

value_counts(data['Alone'])

## Encode Sex

In [ ]:
data['Sex'] = LabelEncoder().fit_transform(data['Sex'])

value_counts(data['Sex'])

## Extract Title from Name

In [ ]:
data['Title'] = data['Name'].str.extract(r', (.*)\.', expand=False)

value_counts(data['Title'])

See [english honorifics](https://en.wikipedia.org/wiki/English_honorifics) for reference.

* TODO: fix regexp for `Mrs. Martin (Elizabeth L` and `the Countess`

In [ ]:
data['Title'].replace(['Mlle', 'Ms'], 'Miss', inplace=True)
data['Title'].replace(['Mme', 'Lady', 'Countess', 'Dona', 'Mrs. Martin (Elizabeth L', 'the Countess'], 'Mrs', inplace=True)
data['Title'].replace(['Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer'], 'Mr', inplace=True)
data['Title'] = LabelEncoder().fit_transform(data['Title'])
value_counts(data['Title'])

## Fix Fare

In [ ]:
def infer(df:pd.DataFrame, features:np.array, target:str):
    train = df[df[target].notna()]
    
    # select training data and fit regressor
    x = train.loc[:, features]
    y = train.loc[:, target]
    reg = linear_model.LinearRegression().fit(x, y)
    print('     score', reg.score(x, y))
    
    pred = reg.predict(x)
    print('       mae', metrics.mean_absolute_error(y, pred))
    
    # check prediction is better than mean
    print('  mean mae', metrics.mean_absolute_error(y, np.full((train.shape[0]), y.mean())))
    
    # check prediction is better than median
    print('median mae', metrics.mean_absolute_error(y, np.full((train.shape[0]), y.median())))
    
    # predict missing target values
    na_mask = df[target].isna()
    predict = df[na_mask]
    x_predict = predict.loc[:, features]
    y_predict = reg.predict(x_predict)

    # create new feature
    df[target + '_'] = df[target]
    df.loc[na_mask, target + '_'] = y_predict
    
infer(data, ['Pclass', 'SibSp', 'Title'], 'Fare')
# Sex and Parch - increase mae

count(data)

## Fix Age

In [ ]:
infer(data, ['Pclass', 'FamilySize', 'Title', 'Fare_'], 'Age')
# Sex

count(data)

## Encode and fix Embarked

In [ ]:
def encode_labels(df:pd.DataFrame, target: str):
    notna = df[target].notna()
    y = df[notna].loc[:, target]
    df.loc[notna, target] = LabelEncoder().fit_transform(y)

encode_labels(data, 'Embarked')
infer(data, ['Pclass', 'SibSp', 'Parch', 'Title', 'Fare_', 'Age_'], 'Embarked')

value_counts(data['Embarked_'])

## Finally predict Survived

In [ ]:
train = data[data['Survived'].notna()]
test = data[data['Survived'].isna()]

# prepare train for fitting
# other features appears useless
x = train.loc[:, ['Age_', 'Sex', 'Pclass', 'Fare_', 'FamilySize', 'Alone']]
y = train.loc[:, 'Survived']
XTrain, XValid, YTrain, YValid = train_test_split(x, y, test_size=0.2, random_state=40)

# Fit logistic regression using scikit
LR = linear_model.LogisticRegression(C=1000, solver='lbfgs', max_iter=1000)
LR.fit(X=XTrain, y=YTrain)

def accuracy(Y: np.array, yPred: np.array) -> float:
  return np.sum(yPred==Y) / len(Y)

# Use model to predict on training and validation sets
print('     Train accuracy', accuracy(YTrain, LR.predict(XTrain)))
print('Validation accuracy', accuracy(YValid, LR.predict(XValid)))

## Save results

In [ ]:
# Predict for test set
# Create a Kaggle submission
XTest = test.loc[:, features]
YTest = LR.predict(XTest)
#data['Survivd'] = data['Survived'].astype('int64')
sub = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived': YTest.astype('int64')})
sub.to_csv('submission.csv', index=False)

## Thanks!!!